In [1]:
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd 
from transformers import BertTokenizer, BertModel
import torch

'''print("tensorflow importation")
import tensorflow as tf
print(tf.__version__)'''
#import tensorflow_text 
#print(tensorflow_text.__version__)
#import tensorflow_hub as hub

'''import keras
print(keras.__version__)'''
import mlflow
import matplotlib.pyplot as plt
import gensim
from gensim.models import Word2Vec

from sklearn.base import BaseEstimator, TransformerMixin

from sklearn.manifold import TSNE
import seaborn as sns
from sentence_transformers import SentenceTransformer

import time
from sklearn.linear_model import LogisticRegression
from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report, hamming_loss, jaccard_score



In [2]:
data = pd.read_csv('Data/df_tags_filtered.csv')
data['combined'] = data['Title'] + " " + data['Body']

display(data)

,Title,Body,Tags,Body_word_count,combined
0,android jetpack navigation bottomnavigationvie...,android jetpack navigation bottomnavigationvie...,android android-architecture-components bottom...,131,android jetpack navigation bottomnavigationvie...
1,jetpack compose button drawable,achieve jetpack compose something button eleva...,android android-jetpack-compose android-compos...,60,jetpack compose button drawable achieve jetpac...
2,topappbar flash navigate compose navigation,screen scaffold topappbar navigate jetpack nav...,android android-jetpack android-jetpack-compos...,198,topappbar flash navigate compose navigation sc...
3,how create recycler view compose jetpack,special way create recyclerview compose jetpac...,android android-recyclerview android-jetpack-c...,7,how create recycler view compose jetpack speci...
4,how navhostfragment,integrate android navigation architecture comp...,android android-fragments android-architecture...,48,how navhostfragment integrate android navigati...
...,...,...,...,...,...
33574,spring rest json can handle managedback refere...,post httplocalhosttranslators angularjs fronte...,java json hibernate jackson spring-boot,273,spring rest json can handle managedback refere...
33575,disableenable submit button form fill,form submit button disabledenabled depend form...,javascript forms submit disabled-control,56,disableenable submit button form fill form sub...
33576,how keep net console app run,consider console application start service sep...,c# multithreading sleep manualresetevent,53,how keep net console app run consider console ...
33577,how avoid javalangnoclassdeffounderror,code add text exist doc file itll save another...,java apache-poi noclassdeffounderror doc,165,how avoid javalangnoclassdeffounderror code ad...


In [3]:
def words_tokenize(text):
  text = text.split()
  return text

data['Body'] = data.Body.apply(lambda text: words_tokenize(text))
data['Title'] = data.Title.apply(lambda text: words_tokenize(text))
data['Tags'] = data.Tags.apply(lambda text: words_tokenize(text))
display(data)

,Title,Body,Tags,Body_word_count,combined
0,"[android, jetpack, navigation, bottomnavigatio...","[android, jetpack, navigation, bottomnavigatio...","[android, android-architecture-components, bot...",131,android jetpack navigation bottomnavigationvie...
1,"[jetpack, compose, button, drawable]","[achieve, jetpack, compose, something, button,...","[android, android-jetpack-compose, android-com...",60,jetpack compose button drawable achieve jetpac...
2,"[topappbar, flash, navigate, compose, navigation]","[screen, scaffold, topappbar, navigate, jetpac...","[android, android-jetpack, android-jetpack-com...",198,topappbar flash navigate compose navigation sc...
3,"[how, create, recycler, view, compose, jetpack]","[special, way, create, recyclerview, compose, ...","[android, android-recyclerview, android-jetpac...",7,how create recycler view compose jetpack speci...
4,"[how, navhostfragment]","[integrate, android, navigation, architecture,...","[android, android-fragments, android-architect...",48,how navhostfragment integrate android navigati...
...,...,...,...,...,...
33574,"[spring, rest, json, can, handle, managedback,...","[post, httplocalhosttranslators, angularjs, fr...","[java, json, hibernate, jackson, spring-boot]",273,spring rest json can handle managedback refere...
33575,"[disableenable, submit, button, form, fill]","[form, submit, button, disabledenabled, depend...","[javascript, forms, submit, disabled-control]",56,disableenable submit button form fill form sub...
33576,"[how, keep, net, console, app, run]","[consider, console, application, start, servic...","[c#, multithreading, sleep, manualresetevent]",53,how keep net console app run consider console ...
33577,"[how, avoid, javalangnoclassdeffounderror]","[code, add, text, exist, doc, file, itll, save...","[java, apache-poi, noclassdeffounderror, doc]",165,how avoid javalangnoclassdeffounderror code ad...


In [4]:
X = data[['Title','Body']]
y = data['Tags']
X_test = pd.read_csv("Data/X_test.csv")
y_test = pd.read_csv("Data/y_test.csv")
display(y)

from sklearn.model_selection import train_test_split
df = pd.DataFrame(data)
def merge_and_keep_first_occurrence(row):
    merged = row['Title'] + row['Body']
    return list(dict.fromkeys(merged))
# Appliquer la fonction à chaque ligne du DataFrame
df['Merged'] = data.apply(merge_and_keep_first_occurrence, axis=1)

X = df['Merged']
display(X)

X_train, X_test, y_train, y_test = train_test_split(X,y,
    test_size=0.2,
    random_state=42
)


display(X_train.shape, X_test.shape)
X_train_sample = X_train.sample(8000, random_state=42)
y_train_sample = y_train.loc[X_train_sample.index]
X_test_sample = X_test.sample(100, random_state=42)
y_test_sample = y_test.loc[X_test_sample.index]

0        [android, android-architecture-components, bot...
1        [android, android-jetpack-compose, android-com...
2        [android, android-jetpack, android-jetpack-com...
3        [android, android-recyclerview, android-jetpac...
4        [android, android-fragments, android-architect...
                               ...                        
33574        [java, json, hibernate, jackson, spring-boot]
33575        [javascript, forms, submit, disabled-control]
33576        [c#, multithreading, sleep, manualresetevent]
33577        [java, apache-poi, noclassdeffounderror, doc]
33578        [java, jetty, noclassdeffounderror, shutdown]
Name: Tags, Length: 33579, dtype: object

0        [android, jetpack, navigation, bottomnavigatio...
1        [jetpack, compose, button, drawable, achieve, ...
2        [topappbar, flash, navigate, compose, navigati...
3        [how, create, recycler, view, compose, jetpack...
4        [how, navhostfragment, integrate, android, nav...
                               ...                        
33574    [spring, rest, json, can, handle, managedback,...
33575    [disableenable, submit, button, form, fill, di...
33576    [how, keep, net, console, app, run, consider, ...
33577    [how, avoid, javalangnoclassdeffounderror, cod...
33578    [jetty, throw, noclassdeffounderror, orgeclips...
Name: Merged, Length: 33579, dtype: object

(26863,)

(6716,)

In [5]:
print(y_test_sample)

10229    [javascript, jquery, inheritance, prototypal-i...
4932     [javascript, asp.net-mvc, json, asp.net-mvc-4,...
6712     [python, django, google-app-engine, web-applic...
29503      [javascript, html, angularjs, validation, grid]
3643     [android, android-widget, android-5.0-lollipop...
                               ...                        
1889     [javascript, reactjs, client-side, react-route...
5621     [sql-server, sql-server-2005, database-design,...
31108       [java, drawing, graphviz, graph-visualization]
9165     [sql, sql-server-2008, excel, copy-paste, sql-...
8178     [javascript, jquery, ecmascript-6, browserify,...
Name: Tags, Length: 100, dtype: object


## Pipeline vectorisation

In [7]:


    
class CustomTfidfVectorizer(BaseEstimator, TransformerMixin):
    def __init__(self, min_df=0.008, max_df=0.9, ngram_range=(1, 1), norm='l2', use_idf=True, stop_words='english'):
        self.min_df = min_df
        self.max_df = max_df
        self.ngram_range = ngram_range
        self.norm = norm
        self.use_idf = use_idf
        self.stop_words = stop_words
        self.vectorizer = TfidfVectorizer(min_df=self.min_df, max_df=self.max_df,
                                          ngram_range=self.ngram_range, norm=self.norm,
                                          use_idf=self.use_idf, stop_words=self.stop_words)

    def fit(self, X, y=None):
        self.vectorizer.fit(X)
        return self

    def transform(self, X, y=None):
        return self.vectorizer.transform(X)
    


from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import MultiLabelBinarizer

class CustomWord2VecVectorizer(BaseEstimator, TransformerMixin):
    def __init__(self, vector_size=400, window=8, min_count=6,alpha=0.025, epochs=5,workers=5, sg=0, max_vocab_size=None):
        self.vector_size = vector_size
        self.window = window
        self.min_count = min_count
        self.alpha = alpha
        self.epochs = epochs
        self.workers = workers
        self.sg = sg
        self.max_vocab_size = max_vocab_size
        self.model = Word2Vec(
            vector_size=self.vector_size, 
            window=self.window, 
            workers=self.workers, 
            sg=self.sg
        )
    def fit(self, X, y=None):
        self.model = Word2Vec(
            vector_size=self.vector_size, 
            window=self.window, 
            min_count=self.min_count,
            alpha=self.alpha,
            workers=self.workers,
            sg=self.sg,
            max_vocab_size=self.max_vocab_size
        )
        self.model.build_vocab(X)
        return self

    def transform(self, X, y=None):
        # Moyenne des vecteurs de mots pour chaque document
        return np.array([np.mean([self.model.wv[w] for w in words if w in self.model.wv]
                                 or [np.zeros(self.model.vector_size)], axis=0)
                         for words in X])




class CustomBERTVectorizer(BaseEstimator, TransformerMixin):
    def __init__(self, max_length=128):
        self.tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
        self.model = BertModel.from_pretrained('bert-base-uncased')
        self.max_length = max_length

    def fit(self, X, y=None):
        return self

    def transform(self, X, y=None):
        # Tokenization
        inputs = self.tokenizer(X, padding=True, truncation=True, max_length=self.max_length, return_tensors="pt")
        
        # Passer les inputs à travers BERT
        with torch.no_grad():
            outputs = self.model(**inputs)
        
        # Récupérer la représentation des tokens CLS (premier token)
        embeddings = outputs.last_hidden_state[:, 0, :].numpy()
        
        return embeddings



class SentenceTransformerVectorizer(BaseEstimator, TransformerMixin):
    def __init__(self, model_name='all-MiniLM-L6-v2'):
        self.model = SentenceTransformer(model_name)

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return self.model.encode(X)
    

def create_tfidf_pipeline():
    pipeline_tfidf = Pipeline([
        
        ("tfidf", TfidfVectorizer(min_df=0.007, max_df=0.9, ngram_range=(1, 1), stop_words='english')),
        #('mlb', MultiLabelBinarizer()),
        ("clf", OneVsRestClassifier(LogisticRegression())),
        #("RandomForest", RandomForestClassifier(n_estimators=100, random_state=42))
    ])
    return pipeline_tfidf

def create_word2vec_pipeline():
    pipeline_word2vec = Pipeline([
        ("word2vec", CustomWord2VecVectorizer()),
        ("clf_w2v", OneVsRestClassifier(LogisticRegression())),

    ])
    return pipeline_word2vec

def create_bert_pipeline():
    pipeline_bert = Pipeline([
        ("bert_vectorizer", CustomBERTVectorizer()),
        ("clf_bert", OneVsRestClassifier(LogisticRegression(max_iter=1000)))
    ])
    return pipeline_bert

def create_use_pipeline():
    pipeline_use = Pipeline([
        ("use_vectorizer", SentenceTransformerVectorizer()),
        ("clf_use", OneVsRestClassifier(LogisticRegression(max_iter=1000, C=1)))
    ])
    return pipeline_use



In [8]:
def stocker_results(name = str, precision_test=None, jaccard_test=None, fit_time=None, transform_time=None):
    # Enregistrer les résultats dans un DataFrame
    results = pd.DataFrame({'Vectorizer': [name], 'precision Test': [precision_test], 'Jaccard Test': [jaccard_test], 'Fit Time': [fit_time]})
    display(results)
    return results

In [9]:
print(y_train_sample[:10])

342      [android, user-interface, android-support-libr...
11837    [c#, asp.net-mvc, linq, entity-framework, asp....
31584       [java, collections, null, java-8, java-stream]
14632    [javascript, jquery, script-tag, javascript-in...
24015    [css, svg, reactjs, material-design, material-ui]
19968    [c#, asp.net, google-chrome, listbox, autopost...
8458     [c#, visual-studio, visual-studio-2013, intell...
20386    [ios, xcode, label, nsattributedstring, center...
15009    [sql, sql-server, sql-server-2008, stored-proc...
10601    [c#, winforms, datagridview, datatable, backgr...
Name: Tags, dtype: object


In [10]:
mlb = MultiLabelBinarizer()

# Adapter MultiLabelBinarizer et transformer les étiquettes
y_train_sample_trans = mlb.fit_transform(y_train_sample)
y_test_sample_trans = mlb.transform(y_test_sample) 


/Users/bahia/opt/anaconda3/lib/python3.9/site-packages/sklearn/preprocessing/_label.py:900: UserWarning: unknown class(es) ['android-cursor', 'application-pool', 'browserify', 'c#-2.0', 'client-side-scripting', 'cobertura', 'convenience-methods', 'custom-errors', 'drawing', 'e2e-testing', 'eclipse-pde', 'filesystemobject', 'fullcalendar-3', 'gesture-recognition', 'google-api-objc-client', 'graph-visualization', 'heredoc', 'icloneable', 'insertafter', 'kafka-consumer-api', 'kafka-python', 'latex', 'messagebox', 'micrometer-tracing', 'moss', 'project-server', 'shared-objects', 'street-address', 'system-shutdown', 'tipfy', 'web-parts', 'xmlexception', 'zipkin'] will be ignored
  warnings.warn(


## Vectorisation avec TF_IDF.
  Régréssion logistique:  

  RandomForest:  

In [11]:
X_train_sample_str = [' '.join(text) for text in X_train_sample]

In [12]:
from sklearn.model_selection import GridSearchCV


'''pipeline_tfidf_grid = create_tfidf_pipeline()
param_grid_tfidf = {
    'tfidf__min_df': [1, 5, 10],
    'tfidf__max_df': [0.5, 0.75, 1.0],
    'tfidf__ngram_range': [(1, 1), (1, 2), (1, 3)],
    'tfidf__norm': ['l1', 'l2'],
    'tfidf__use_idf': [True, False],
    
}

grid_search = GridSearchCV(
    pipeline_tfidf_grid, param_grid=param_grid_tfidf, 
    cv=5,scoring=['f1_weighted'] ,verbose=1, refit='f1_weighted')

# Convertir la matrice CSR en une liste de listes de mots
#X_train_sample_trans_list = [row.nonzero()[1].tolist() for row in X_train_sample]

# Entraîner le modèle Word2Vec
grid_search.fit(X_train_sample_str, y_train_sample_trans)

#grid_search.fit(X_train_sample_trans, y_train_sample_trans)

print("Meilleurs hyperparamètres : ", grid_search.best_params_)'''

'pipeline_tfidf_grid = create_tfidf_pipeline()\nparam_grid_tfidf = {\n    \'tfidf__min_df\': [1, 5, 10],\n    \'tfidf__max_df\': [0.5, 0.75, 1.0],\n    \'tfidf__ngram_range\': [(1, 1), (1, 2), (1, 3)],\n    \'tfidf__norm\': [\'l1\', \'l2\'],\n    \'tfidf__use_idf\': [True, False],\n    \n}\n\ngrid_search = GridSearchCV(\n    pipeline_tfidf_grid, param_grid=param_grid_tfidf, \n    cv=5,scoring=[\'f1_weighted\'] ,verbose=1, refit=\'f1_weighted\')\n\n# Convertir la matrice CSR en une liste de listes de mots\n#X_train_sample_trans_list = [row.nonzero()[1].tolist() for row in X_train_sample]\n\n# Entraîner le modèle Word2Vec\ngrid_search.fit(X_train_sample_str, y_train_sample_trans)\n\n#grid_search.fit(X_train_sample_trans, y_train_sample_trans)\n\nprint("Meilleurs hyperparamètres : ", grid_search.best_params_)'

In [13]:
pipeline_tfidf = create_tfidf_pipeline()

# Convertir les listes en chaînes de caractères
X_train_sample_str = [' '.join(text) for text in X_train_sample]
X_test_sample_str = [' '.join(text) for text in X_test_sample]

# Utiliser les chaînes de caractères pour l'ajustement et la transformation
X_train_sample_trans = pipeline_tfidf["tfidf"].fit_transform(X_train_sample_str)
X_test_sample_trans = pipeline_tfidf["tfidf"].transform(X_test_sample_str)


In [14]:
print(X_train_sample_trans.shape, X_test_sample_trans.shape)
print(y_test_sample_trans.shape, y_train_sample_trans.shape)

(8000, 986) (100, 986)
(100, 5798) (8000, 5798)


In [15]:
'''print(y_test_sample_trans[:100])
print(y_train_sample_trans)
print(X_train_sample_trans)
print(X_train_sample_trans)'''

'print(y_test_sample_trans[:100])\nprint(y_train_sample_trans)\nprint(X_train_sample_trans)\nprint(X_train_sample_trans)'

### Regression logistique OneVsRestClassifier

In [16]:
lr_tfidf = pipeline_tfidf.named_steps['clf'].fit(X_train_sample_trans, y_train_sample_trans)

predictions = lr_tfidf.predict_proba(X_test_sample_trans)
print("shape de prediction",predictions.shape)
print("shape de prediction",X_test_sample_trans.shape)


shape de prediction (100, 5798)
shape de prediction (100, 986)


In [17]:
probabilities = predictions

# Définir un seuil personnalisé pour sélectionner les tags
threshold = 0.15  # baisser le seuil pour inclure plus de tags

def select_top_n_tags(probabilities, threshold, top_n=5):
    # Créer une matrice de zéros de la même forme que probabilities
    final_tags = np.zeros(probabilities.shape)
    
    for i in range(probabilities.shape[0]):
        # Trouver les indices qui satisfont le seuil
        eligible_indices = np.where(probabilities[i] >= threshold)[0]
        
        # Trier ces indices en fonction de leur probabilité
        top_indices = eligible_indices[np.argsort(probabilities[i][eligible_indices])][::-1]
        
        # Sélectionner les top N indices
        top_n_indices = top_indices[:top_n]
        
        # Mettre à 1 les positions correspondant aux tags sélectionnés
        final_tags[i, top_n_indices] = 1
    
    return final_tags

# Utiliser la fonction pour obtenir les tags prédits

binary_predictions = select_top_n_tags(probabilities, threshold, top_n=7)

Hamming Loss : C'est la fraction des labels mal prédits par rapport à l'ensemble total de labels. C'est une métrique appropriée pour la classification multilabels car elle prend en compte chaque label individuellement.

Jaccard Score : C'est la taille de l'intersection des labels prédits et des vrais labels divisée par la taille de l'union des labels prédits et des vrais labels. C'est une bonne métrique pour la classification multilabels car elle prend en compte la présence et l'absence de chaque label.

Precision, Recall, F1 Score : Ces métriques peuvent également être utilisées dans le contexte multilabels, en calculant la métrique pour chaque label individuellement, puis en prenant la moyenne.

In [18]:
print("Shape of true values:", y_test_sample_trans.shape)
print("Shape of predictions:", binary_predictions.shape)

Shape of true values: (100, 5798)
Shape of predictions: (100, 5798)


In [19]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report, hamming_loss, jaccard_score


print("Precision:", precision_score(y_test_sample_trans, binary_predictions, average='samples'))
print("Recall:", recall_score(y_test_sample_trans, binary_predictions, average='samples'))
print("F1 Score:", f1_score(y_test_sample_trans, binary_predictions, average='samples'))



Precision: 0.66
Recall: 0.3518333333333334
F1 Score: 0.431487012987013


/Users/bahia/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [20]:

print("Jaccard Score:", jaccard_score(y_test_sample_trans, binary_predictions, average='samples'))


Jaccard Score: 0.30469047619047623


In [21]:

predictions_labels =mlb.inverse_transform(binary_predictions)
print(y_test_sample[:5])
# Afficher les labels des prédictions
for labels in predictions_labels:
    print(labels)

10229    [javascript, jquery, inheritance, prototypal-i...
4932     [javascript, asp.net-mvc, json, asp.net-mvc-4,...
6712     [python, django, google-app-engine, web-applic...
29503      [javascript, html, angularjs, validation, grid]
3643     [android, android-widget, android-5.0-lollipop...
Name: Tags, dtype: object
('c#', 'javascript')
('asp.net', 'asp.net-mvc', 'c#', 'javascript')
('android', 'django', 'google-app-engine', 'python')
('html', 'javascript')
('android',)
('javascript',)
('ios', 'objective-c')
('c#',)
('python',)
('sql', 'sql-server', 'sql-server-2008', 't-sql')
('android', 'android-gradle-plugin', 'gradle')
('python',)
('ios', 'javascript', 'react-native', 'xcode')
('android', 'python')
('android', 'c#', 'java', 'python')
('c#', 'java')
('java',)
('c++', 'switch-statement')
('javascript',)
('ios',)
('html', 'javascript')
('arrays', 'python', 'string')
('c#', 'java')
('javascript', 'python')
('datetime', 'ios', 'iphone', 'objective-c', 'python')
('c#', 'c++', 'visual-

In [22]:
# Calculer le nombre total de tags prédits
total_predicted_tags = binary_predictions.sum()

# Calculer le nombre total de tags possibles
total_possible_tags = y_test_sample_trans.size

# Calculer le taux de remplissage
fill_rate = total_predicted_tags / total_possible_tags

print(f'Taux de remplissage des tags : {fill_rate * 100:.2f}%')

Taux de remplissage des tags : 0.04%


In [23]:
result_lr_tfidf = stocker_results(name="lr_Tfidf", 
                                 precision_test=precision_score(y_test_sample_trans, binary_predictions, average='samples'),
                                 jaccard_test=jaccard_score(y_test_sample_trans, binary_predictions, average='samples'))

/Users/bahia/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


,Vectorizer,precision Test,Jaccard Test,Fit Time
0,lr_Tfidf,0.66,0.30469,None


In [24]:
'''import joblib

# Sauvegarde du modèle entraîné dans un fichier .pkl
model_path = 'final_model.pkl'
joblib.dump(lr_tfidf, model_path)'''

"import joblib\n\n# Sauvegarde du modèle entraîné dans un fichier .pkl\nmodel_path = 'final_model.pkl'\njoblib.dump(lr_tfidf, model_path)"

### Random Forest

In [25]:
#rf_tfidf = pipeline_tfidf.named_steps['RandomForest'].fit(X_train_sample_trans, y_train_sample_trans)

In [26]:
#predictions = rf_tfidf.predict(X_test_sample_trans)

In [27]:
'''probabilities = predictions
# Définir un seuil personnalisé pour sélectionner les tags
threshold = 0.2  # baisser le seuil pour inclure plus de tags

# Utiliser la fonction pour obtenir les tags prédits
rf_binary_predictions = select_top_n_tags(probabilities, threshold, top_n=5)'''

'probabilities = predictions\n# Définir un seuil personnalisé pour sélectionner les tags\nthreshold = 0.2  # baisser le seuil pour inclure plus de tags\n\n# Utiliser la fonction pour obtenir les tags prédits\nrf_binary_predictions = select_top_n_tags(probabilities, threshold, top_n=5)'

In [28]:
'''print("Precision:", precision_score(y_test_sample_trans, rf_binary_predictions, average='samples'))
print("Recall:", recall_score(y_test_sample_trans, rf_binary_predictions, average='samples'))
print("F1 Score:", f1_score(y_test_sample_trans, rf_binary_predictions, average='samples'))
rf_predictions_labels =mlb.inverse_transform(rf_binary_predictions)
print(y_test_sample[:5])
# Afficher les labels des prédictions
for labels in rf_predictions_labels:
    print(labels)
'''

'print("Precision:", precision_score(y_test_sample_trans, rf_binary_predictions, average=\'samples\'))\nprint("Recall:", recall_score(y_test_sample_trans, rf_binary_predictions, average=\'samples\'))\nprint("F1 Score:", f1_score(y_test_sample_trans, rf_binary_predictions, average=\'samples\'))\nrf_predictions_labels =mlb.inverse_transform(rf_binary_predictions)\nprint(y_test_sample[:5])\n# Afficher les labels des prédictions\nfor labels in rf_predictions_labels:\n    print(labels)\n'

In [29]:
#result_rf_tfidf = stocker_results(name="rf_Tfidf",precision_test=precision_score(y_test_sample_trans, rf_binary_predictions,average='samples'),
#                                 jaccard_test=jaccard_score(y_test_sample_trans, rf_binary_predictions, average='samples'))

In [30]:
import matplotlib.pyplot as plt

'''# Assurez-vous que vos DataFrames sont concaténés en un seul DataFrame
results = pd.concat([result_lr_tfidf, result_rf_tfidf])

fig, axs = plt.subplots(1, 2, figsize=(15, 6))  # 1 ligne, 2 colonnes

# Sous-graphique pour la précision
axs[0].bar(results['Vectorizer'], results['precision Test'], width=0.4)
axs[0].set_title('Comparaison de la précision')
axs[0].set_xlabel('Modèle')
axs[0].set_ylabel('Précision')

# Sous-graphique pour le score Jaccard
axs[1].bar(results['Vectorizer'], results['Jaccard Test'], width=0.4)
axs[1].set_title('Comparaison du score Jaccard')
axs[1].set_xlabel('Modèle')
axs[1].set_ylabel('Score Jaccard')

plt.tight_layout()
plt.show()'''

"# Assurez-vous que vos DataFrames sont concaténés en un seul DataFrame\nresults = pd.concat([result_lr_tfidf, result_rf_tfidf])\n\nfig, axs = plt.subplots(1, 2, figsize=(15, 6))  # 1 ligne, 2 colonnes\n\n# Sous-graphique pour la précision\naxs[0].bar(results['Vectorizer'], results['precision Test'], width=0.4)\naxs[0].set_title('Comparaison de la précision')\naxs[0].set_xlabel('Modèle')\naxs[0].set_ylabel('Précision')\n\n# Sous-graphique pour le score Jaccard\naxs[1].bar(results['Vectorizer'], results['Jaccard Test'], width=0.4)\naxs[1].set_title('Comparaison du score Jaccard')\naxs[1].set_xlabel('Modèle')\naxs[1].set_ylabel('Score Jaccard')\n\nplt.tight_layout()\nplt.show()"

## Embedding avec word2Vec

In [31]:
# Instancier le modèle avec le préprocesseur dans le pipeline
from sklearn.model_selection import GridSearchCV


'''pipeline_w2v_grid = create_word2vec_pipeline()

# Définir la grille des hyperparamètres à rechercher
param_grid_w2v = {
    'word2vec__vector_size': [100, 200, 300, 400],
    'word2vec__window': [5, 10, 15],
    'word2vec__min_count': [1, 2, 5],
    'word2vec__alpha': [0.01, 0.025, 0.05],
    'word2vec__epochs': [5, 10, 20],
    'word2vec__workers': [3, 5, 8],
    'word2vec__sg': [0, 1],
    'word2vec__max_vocab_size': [None, 10000, 20000]
}

grid_search = GridSearchCV(
    pipeline_w2v_grid, param_grid=param_grid_w2v, 
    cv=5,scoring=['f1_micro','precision_micro'], refit='f1_micro')

# Convertir la matrice CSR en une liste de listes de mots
X_train_sample_trans_list = [row.nonzero()[1].tolist() for row in X_train_sample_trans]

# Entraîner le modèle Word2Vec
grid_search.fit(X_train_sample_trans_list, y_train_sample_trans)

#grid_search.fit(X_train_sample_trans, y_train_sample_trans)

print("Meilleurs hyperparamètres : ", grid_search.best_params_)'''

'pipeline_w2v_grid = create_word2vec_pipeline()\n\n# Définir la grille des hyperparamètres à rechercher\nparam_grid_w2v = {\n    \'word2vec__vector_size\': [100, 200, 300, 400],\n    \'word2vec__window\': [5, 10, 15],\n    \'word2vec__min_count\': [1, 2, 5],\n    \'word2vec__alpha\': [0.01, 0.025, 0.05],\n    \'word2vec__epochs\': [5, 10, 20],\n    \'word2vec__workers\': [3, 5, 8],\n    \'word2vec__sg\': [0, 1],\n    \'word2vec__max_vocab_size\': [None, 10000, 20000]\n}\n\ngrid_search = GridSearchCV(\n    pipeline_w2v_grid, param_grid=param_grid_w2v, \n    cv=5,scoring=[\'f1_micro\',\'precision_micro\'], refit=\'f1_micro\')\n\n# Convertir la matrice CSR en une liste de listes de mots\nX_train_sample_trans_list = [row.nonzero()[1].tolist() for row in X_train_sample_trans]\n\n# Entraîner le modèle Word2Vec\ngrid_search.fit(X_train_sample_trans_list, y_train_sample_trans)\n\n#grid_search.fit(X_train_sample_trans, y_train_sample_trans)\n\nprint("Meilleurs hyperparamètres : ", grid_search.b

In [32]:
pipeline_w2v = create_word2vec_pipeline()
# Convertir les listes en chaînes de caractères
X_train_sample_str = [' '.join(text) for text in X_train_sample]
X_test_sample_str = [' '.join(text) for text in X_test_sample]

# Utiliser les chaînes de caractères pour l'ajustement et la transformation
X_train_sample_trans_w2v = pipeline_w2v['word2vec'].fit_transform(X_train_sample_str)
X_test_sample_trans_w2v = pipeline_w2v['word2vec'].transform(X_test_sample_str)

mlb2 = MultiLabelBinarizer()
y_train_sample_trans = mlb2.fit_transform(y_train_sample)

In [33]:
print(X_train_sample_trans_w2v.shape)
print(X_test_sample_trans_w2v.shape)

(8000, 400)
(100, 400)


In [34]:
lr_w2v = pipeline_w2v.named_steps['clf_w2v'].fit(X_train_sample_trans_w2v, y_train_sample_trans)
print(lr_w2v)
predictions_w2v = lr_w2v.predict_proba(X_test_sample_trans_w2v)
print("shape de prediction",predictions_w2v.shape)
print("shape de X_test_sample_trans",X_test_sample_trans_w2v.shape)



OneVsRestClassifier(estimator=LogisticRegression())
shape de prediction (100, 5798)
shape de X_test_sample_trans (100, 400)


In [35]:
probabilities = predictions_w2v
# Définir un seuil personnalisé pour sélectionner les tags
threshold = 0.15  # baisser le seuil pour inclure plus de tags

# Utiliser la fonction pour obtenir les tags prédits
rf_w2v_binary_predictions = select_top_n_tags(probabilities, threshold, top_n=5)
result_lr_w2f = stocker_results(name="lr_w2v",precision_test=precision_score(y_test_sample_trans, rf_w2v_binary_predictions,average='samples'),
                                 jaccard_test=jaccard_score(y_test_sample_trans, rf_w2v_binary_predictions, average='samples'))

,Vectorizer,precision Test,Jaccard Test,Fit Time
0,lr_w2v,0.175,0.069333,None


In [36]:
print("Precision:", precision_score(y_test_sample_trans, rf_w2v_binary_predictions, average='samples'))
print("Recall:", recall_score(y_test_sample_trans, rf_w2v_binary_predictions, average='samples'))
print("F1 Score:", f1_score(y_test_sample_trans, rf_w2v_binary_predictions, average='samples'))

Precision: 0.175
Recall: 0.08716666666666667
F1 Score: 0.11533333333333333


In [37]:
# Afficher les labels des prédictions
predictions_labels =mlb.inverse_transform(rf_w2v_binary_predictions)
print(predictions_labels)
print(y_test_sample[:5])


[('android', 'javascript'), ('android', 'javascript'), ('android', 'javascript'), ('android', 'javascript'), ('android', 'javascript'), ('android', 'javascript'), ('android', 'javascript'), ('android', 'javascript'), ('android', 'javascript'), ('android', 'javascript'), ('android', 'javascript'), ('android', 'javascript'), ('android', 'javascript'), ('android', 'javascript'), ('android', 'javascript'), ('android', 'javascript'), ('android', 'javascript'), ('android', 'javascript'), ('android', 'javascript'), ('android', 'javascript'), ('android', 'javascript'), ('android', 'javascript'), ('android', 'javascript'), ('android', 'javascript'), ('android', 'javascript'), ('android', 'javascript'), ('android', 'javascript'), ('android', 'javascript'), ('android', 'javascript'), ('android', 'javascript'), ('android', 'javascript'), ('android', 'javascript'), ('android', 'javascript'), ('android', 'javascript'), ('android', 'javascript'), ('android', 'javascript'), ('android', 'javascript'), 

### BERT Embedding

"Bidirectional Encoder Representations from Transformers":  

BERT est un modèle de langage bidirectionnel basé sur des transformers, capable de capturer les relations complexes entre les mots en utilisant le contexte gauche-droite et droite-gauche pour générer des embeddings riches en informations pour chaque mot.

In [38]:
pipeline_bert = create_bert_pipeline()
# Convertir les listes en chaînes de caractères
X_train_sample_str = [' '.join(text) for text in X_train_sample]
X_test_sample_str = [' '.join(text) for text in X_test_sample]

# Utiliser les chaînes de caractères pour l'ajustement et la transformation
X_train_sample_trans_bert = pipeline_bert['bert_vectorizer'].fit_transform(X_train_sample_str)
X_test_sample_trans_bert = pipeline_bert['bert_vectorizer'].transform(X_test_sample_str)
 
mlb3 = MultiLabelBinarizer()
y_train_sample_trans = mlb3.fit_transform(y_train_sample)

In [ ]:
from sklearn.model_selection import GridSearchCV


'''pipeline_bert_grid = create_bert_pipeline()


param_grid_bert = {
    'bert_vectorizer__max_length': [64, 128, 256],
    'bert_vectorizer__model_name': ['all-MiniLM-L6-v2', 'all-mpnet-base-v2', 'all-distilroberta-v1'],
    'clf_bert__estimator__C': [0.1, 1, 10],
    'clf_bert__estimator__max_iter': [1000, 2000, 3000],


}


grid_search = GridSearchCV(
    pipeline_bert_grid, param_grid=param_grid_bert, 
    cv=5,scoring=['f1_micro'], refit='f1_micro')

# Convertir la matrice CSR en une liste de listes de mots
X_train_sample_trans_list = [row.nonzero()[1].tolist() for row in X_train_sample_trans]

# Entraîner le modèle Word2Vec
grid_search.fit(X_train_sample_trans_list, y_train_sample_trans)

#grid_search.fit(X_train_sample_trans, y_train_sample_trans)

print("Meilleurs hyperparamètres : ", grid_search.best_params_)'''

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [ ]:
pipeline_bert = create_bert_pipeline()
# Convertir les listes en chaînes de caractères
X_train_sample_str = [' '.join(text) for text in X_train_sample]
X_test_sample_str = [' '.join(text) for text in X_test_sample]

# Utiliser les chaînes de caractères pour l'ajustement et la transformation
X_train_sample_trans_bert = pipeline_bert['bert_vectorizer'].fit_transform(X_train_sample_str)
X_test_sample_trans_bert = pipeline_bert['bert_vectorizer'].transform(X_test_sample_str)
 
mlb3 = MultiLabelBinarizer()
y_train_sample_trans = mlb3.fit_transform(y_train_sample)

In [ ]:
lr_bert = pipeline_bert.named_steps['clf_bert'].fit(X_train_sample_trans_bert, y_train_sample_trans)
print(lr_bert)
predictions_bert = lr_bert.predict_proba(X_test_sample_trans_bert)
print("shape de prediction",predictions_bert.shape)
print("shape de X_test_sample_trans",X_test_sample_trans_bert.shape)

binary_pred_bert =select_top_n_tags(probabilities, threshold, top_n=5)
print("Precision:", precision_score(y_test_sample_trans, binary_pred_bert, average='samples'))
print("Recall:", recall_score(y_test_sample_trans, binary_pred_bert, average='samples'))
print("F1 Score:", f1_score(y_test_sample_trans, binary_pred_bert, average='samples'))


'lr_bert = pipeline_bert.named_steps[\'clf_bert\'].fit(X_train_sample_trans_bert, y_train_sample_trans)\nprint(lr_bert)\npredictions_bert = lr_bert.predict_proba(X_test_sample_trans_bert)\nprint("shape de prediction",predictions_bert.shape)\nprint("shape de X_test_sample_trans",X_test_sample_trans_bert.shape)\n\nbinary_pred_bert =select_top_n_tags(probabilities, threshold, top_n=5)\nprint("Precision:", precision_score(y_test_sample_trans, binary_pred_bert, average=\'samples\'))\nprint("Recall:", recall_score(y_test_sample_trans, binary_pred_bert, average=\'samples\'))\nprint("F1 Score:", f1_score(y_test_sample_trans, binary_pred_bert, average=\'samples\'))'

## USE embbedding

In [ ]:
'''pipeline_use = create_use_pipeline()
# Convertir les listes en chaînes de caractères
X_train_sample_str = [' '.join(text) for text in X_train_sample]
X_test_sample_str = [' '.join(text) for text in X_test_sample]

# Utiliser les chaînes de caractères pour l'ajustement et la transformation
X_train_sample_trans_use = pipeline_use['use_vectorizer'].fit_transform(X_train_sample_str)
X_test_sample_trans_use = pipeline_use['use_vectorizer'].transform(X_test_sample_str)
 
mlb3 = MultiLabelBinarizer()
y_train_sample_trans = mlb3.fit_transform(y_train_sample)'''

"pipeline_use = create_use_pipeline()\n# Convertir les listes en chaînes de caractères\nX_train_sample_str = [' '.join(text) for text in X_train_sample]\nX_test_sample_str = [' '.join(text) for text in X_test_sample]\n\n# Utiliser les chaînes de caractères pour l'ajustement et la transformation\nX_train_sample_trans_use = pipeline_use['use_vectorizer'].fit_transform(X_train_sample_str)\nX_test_sample_trans_use = pipeline_use['use_vectorizer'].transform(X_test_sample_str)\n \nmlb3 = MultiLabelBinarizer()\ny_train_sample_trans = mlb3.fit_transform(y_train_sample)"